In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from utils import bn_update, moving_average, copy_model

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(8)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(8, 2)
    def forward(self, x):
        return self.fc(self.pool(self.bn1(self.conv1(x))).view(x.size(0), -1))

In [2]:
model = nn.DataParallel(Model().cuda())
swa_model = nn.DataParallel(Model().cuda())

In [6]:
model.module.fc.weight

Parameter containing:
tensor([[ 0.0019,  0.3173, -0.1361,  0.1433, -0.1931, -0.2400, -0.0885,  0.0504],
        [-0.1701,  0.0804, -0.0250,  0.1087,  0.2881,  0.2766,  0.0864, -0.3460]],
       device='cuda:0', requires_grad=True)

In [7]:
swa_model.module.fc.weight

Parameter containing:
tensor([[ 0.1641, -0.2650, -0.1659, -0.3285, -0.0103,  0.3482,  0.2182,  0.1476],
        [-0.1733, -0.3198, -0.1745, -0.2578, -0.1778, -0.1134,  0.3263,  0.2625]],
       device='cuda:0', requires_grad=True)

In [8]:
SWA_START = 5
SWA_FREQ = 5

In [9]:
for cycle in range(2):
    for epoch in range(20):
            if (epoch+1) == SWA_START:
                copy_model(swa_model, model)
                swa_n = 0
            if ((epoch+1) >= SWA_START) and ((epoch+1) % SWA_FREQ == 0):
                moving_average(swa_model, model, 1.0 / (swa_n + 1))
                swa_n += 1
                #bn_update(train_loader, swa_model)
                #best_metric = valid_model(logger.info, cfg, swa_model,
                #                          valid_loader, optimizer,
                #                          epoch, None,
                #                          best_metric, True)


In [11]:
model.module.fc.weight

Parameter containing:
tensor([[ 0.0019,  0.3173, -0.1361,  0.1433, -0.1931, -0.2400, -0.0885,  0.0504],
        [-0.1701,  0.0804, -0.0250,  0.1087,  0.2881,  0.2766,  0.0864, -0.3460]],
       device='cuda:0', requires_grad=True)

In [12]:
swa_model.module.fc.weight

Parameter containing:
tensor([[ 0.0019,  0.3173, -0.1361,  0.1433, -0.1931, -0.2400, -0.0885,  0.0504],
        [-0.1701,  0.0804, -0.0250,  0.1087,  0.2881,  0.2766,  0.0864, -0.3460]],
       device='cuda:0', requires_grad=True)